In [9]:
from dotenv import load_dotenv
import os

import pandas as pd
from openweather_api import *
from mongodb_functools import MongoDBManager
from postgresql_functools import PostgreSQLManager, OpenWeather

In [10]:
mongo_manager = MongoDBManager()
postgre_manager = PostgreSQLManager()

2024-05-11 12:25:18,483 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-05-11 12:25:18,484 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-11 12:25:18,488 INFO sqlalchemy.engine.Engine select current_schema()
2024-05-11 12:25:18,488 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-11 12:25:18,489 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-05-11 12:25:18,489 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-05-11 12:25:18,490 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-11 12:25:18,502 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [11]:
load_dotenv(override=True)
API_key = os.getenv('OPEN_WEATHER_API_KEY')
locations = ['Canberra']
country_code = 'AU'
limit = 5
data = []

for city_name in locations:
    city_info = fetch_api_data(build_city_info_url(city_name, limit, API_key))
    store_to_json(city_info, 'CityInfo.json')

    latitudes, longitudes = extract_lat_lon(city_info, country_code)
    weather_info = fetch_weather_info_for_coordinates(latitudes, longitudes, API_key)
    store_to_json(weather_info, 'WeatherInfo.json')

    for i in range(len(weather_info)):
        data.append({
            'Date': build_date_timestamp(weather_info[i]['dt'], weather_info[i]['timezone'], 'date'),
            'Location': weather_info[i]['name'],
            'MinTemp': weather_info[i]['main']['temp_min'],
            'MaxTemp': weather_info[i]['main']['temp_max'],
            'Rainfall': get_rain_info(weather_info[i]),
            'Sunrise': build_date_timestamp(weather_info[i]['sys']['sunrise'], weather_info[i]['timezone'], 'hours'),
            'Sunset': build_date_timestamp(weather_info[i]['sys']['sunset'], weather_info[i]['timezone'], 'hours'),
            'WindGustDir': deg_to_cardinal(weather_info[i]['wind']['deg']),
            'WindGustSpeed': weather_info[i]['wind']['speed'],
            'Cloudiness': weather_info[i]['clouds']['all'],
            'Humidity': weather_info[i]['main']['humidity'],
            'Pressure': weather_info[i]['main']['pressure']
        })

weather_df = pd.DataFrame(data)
weather_df.to_csv('WeatherOpenWeatherAUS.csv', index=False)

In [12]:
document = city_info[0]
mongo_manager.insert_document('city', document, verbose=True)
mongo_manager.find_document('city', {'name': 'Canberra'}, verbose=True)

Inserting document...
Document inserted with ID: 663f478f388d3d522fa9dd41
Finding document...
Document found: {'_id': ObjectId('663f478f388d3d522fa9dd41'), 'name': 'Canberra', 'local_names': {'cu': 'Канбєра', 'el': 'Καμπέρα', 'fr': 'Canberra', 'mn': 'Канберра', 'pl': 'Canberra', 'pt': 'Camberra', 'kn': 'ಕ್ಯಾನ್ಬೆರ', 'ka': 'კანბერა', 'bn': 'ক্যানবেরা', 'mi': 'Kānapera', 'tg': 'Канберра', 'sv': 'Canberra', 'ru': 'Канберра', 'ht': 'Kanbera', 'th': 'แคนเบอร์รา', 'hi': 'कैनबरा', 'si': 'කැන්බරා', 'is': 'Canberra', 'lt': 'Kanbera', 'bg': 'Канбера', 'ko': '캔버라', 'tt': 'Канберра', 'yi': 'קאנבערא', 'uk': 'Канберра', 'en': 'Canberra', 'os': 'Канберрæ', 'so': 'Kanbera', 'ur': 'کینبرا', 'be': 'Канбера', 'zh': '堪培拉', 'sr': 'Канбера', 'mr': 'कॅनबेरा', 'ja': 'キャンベラ', 'de': 'Canberra', 'ky': 'Канберра', 'cv': 'Канберра', 'lv': 'Kanbera', 'oc': 'Canberra', 'hy': 'Կանբեռա', 'am': 'ካንበራ', 'mk': 'Канбера', 'es': 'Canberra', 'ku': 'Kanberra', 'fy': 'Kanberra', 'hu': 'Canberra', 'ar': 'كانبرا', 'ug': 'Kanbérr

{'_id': ObjectId('663f478f388d3d522fa9dd41'),
 'name': 'Canberra',
 'local_names': {'cu': 'Канбєра',
  'el': 'Καμπέρα',
  'fr': 'Canberra',
  'mn': 'Канберра',
  'pl': 'Canberra',
  'pt': 'Camberra',
  'kn': 'ಕ್ಯಾನ್ಬೆರ',
  'ka': 'კანბერა',
  'bn': 'ক্যানবেরা',
  'mi': 'Kānapera',
  'tg': 'Канберра',
  'sv': 'Canberra',
  'ru': 'Канберра',
  'ht': 'Kanbera',
  'th': 'แคนเบอร์รา',
  'hi': 'कैनबरा',
  'si': 'කැන්බරා',
  'is': 'Canberra',
  'lt': 'Kanbera',
  'bg': 'Канбера',
  'ko': '캔버라',
  'tt': 'Канберра',
  'yi': 'קאנבערא',
  'uk': 'Канберра',
  'en': 'Canberra',
  'os': 'Канберрæ',
  'so': 'Kanbera',
  'ur': 'کینبرا',
  'be': 'Канбера',
  'zh': '堪培拉',
  'sr': 'Канбера',
  'mr': 'कॅनबेरा',
  'ja': 'キャンベラ',
  'de': 'Canberra',
  'ky': 'Канберра',
  'cv': 'Канберра',
  'lv': 'Kanbera',
  'oc': 'Canberra',
  'hy': 'Կանբեռա',
  'am': 'ካንበራ',
  'mk': 'Канбера',
  'es': 'Canberra',
  'ku': 'Kanberra',
  'fy': 'Kanberra',
  'hu': 'Canberra',
  'ar': 'كانبرا',
  'ug': 'Kanbérra',
  'fa': 'کان

In [13]:
document = weather_info[0]
mongo_manager.insert_document('weather', document, verbose=True)
mongo_manager.find_document('weather', {'name': 'Canberra'}, verbose=True)

Inserting document...
Document inserted with ID: 663f478f388d3d522fa9dd42
Finding document...
Document found: {'_id': ObjectId('663f478f388d3d522fa9dd42'), 'coord': {'lon': 149.1013, 'lat': -35.2976}, 'weather': [{'id': 500, 'main': 'Rain', 'description': 'light rain', 'icon': '10n'}], 'base': 'stations', 'main': {'temp': 11.58, 'feels_like': 11.22, 'temp_min': 10.14, 'temp_max': 13.05, 'pressure': 1026, 'humidity': 93}, 'visibility': 9000, 'wind': {'speed': 2.57, 'deg': 130}, 'rain': {'1h': 0.65}, 'clouds': {'all': 75}, 'dt': 1715423119, 'sys': {'type': 2, 'id': 2093792, 'country': 'AU', 'sunrise': 1715374108, 'sunset': 1715411494}, 'timezone': 36000, 'id': 2172517, 'name': 'Canberra', 'cod': 200}


{'_id': ObjectId('663f478f388d3d522fa9dd42'),
 'coord': {'lon': 149.1013, 'lat': -35.2976},
 'weather': [{'id': 500,
   'main': 'Rain',
   'description': 'light rain',
   'icon': '10n'}],
 'base': 'stations',
 'main': {'temp': 11.58,
  'feels_like': 11.22,
  'temp_min': 10.14,
  'temp_max': 13.05,
  'pressure': 1026,
  'humidity': 93},
 'visibility': 9000,
 'wind': {'speed': 2.57, 'deg': 130},
 'rain': {'1h': 0.65},
 'clouds': {'all': 75},
 'dt': 1715423119,
 'sys': {'type': 2,
  'id': 2093792,
  'country': 'AU',
  'sunrise': 1715374108,
  'sunset': 1715411494},
 'timezone': 36000,
 'id': 2172517,
 'name': 'Canberra',
 'cod': 200}

In [14]:
weather_info = mongo_manager.find_document('weather', {'name': 'Canberra'}, verbose=False)

structured_data = {
    'Date': build_date_timestamp(weather_info['dt'], weather_info['timezone'], 'date'),
    'Location': weather_info['name'],
    'MinTemp': weather_info['main']['temp_min'],
    'MaxTemp': weather_info['main']['temp_max'],
    'Rainfall': get_rain_info(weather_info),
    'Sunrise': build_date_timestamp(weather_info['sys']['sunrise'], weather_info['timezone'], 'hours'),
    'Sunset': build_date_timestamp(weather_info['sys']['sunset'], weather_info['timezone'], 'hours'),
    'WindGustDir': deg_to_cardinal(weather_info['wind']['deg']),
    'WindGustSpeed': weather_info['wind']['speed'],
    'Cloudiness': weather_info['clouds']['all'],
    'Humidity': weather_info['main']['humidity'],
    'Pressure': weather_info['main']['pressure']
}
weather = OpenWeather(**structured_data)
postgre_manager.add_record(weather)
data = postgre_manager.get_record(OpenWeather, Location='Canberra')
print(data)

2024-05-11 12:25:19,139 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-11 12:25:19,142 INFO sqlalchemy.engine.Engine INSERT INTO "OpenWeather" ("Date", "Location", "MinTemp", "MaxTemp", "Rainfall", "Sunrise", "Sunset", "WindGustDir", "WindGustSpeed", "Cloudiness", "Humidity", "Pressure") VALUES (%(Date)s, %(Location)s, %(MinTemp)s, %(MaxTemp)s, %(Rainfall)s, %(Sunrise)s, %(Sunset)s, %(WindGustDir)s, %(WindGustSpeed)s, %(Cloudiness)s, %(Humidity)s, %(Pressure)s) RETURNING "OpenWeather".id
2024-05-11 12:25:19,143 INFO sqlalchemy.engine.Engine [generated in 0.00093s] {'Date': '2024-05-11', 'Location': 'Canberra', 'MinTemp': 10.14, 'MaxTemp': 13.05, 'Rainfall': 0.65, 'Sunrise': '06:48:28', 'Sunset': '17:11:34', 'WindGustDir': 'SE', 'WindGustSpeed': 2.57, 'Cloudiness': 75, 'Humidity': 93, 'Pressure': 1026}
2024-05-11 12:25:19,151 INFO sqlalchemy.engine.Engine COMMIT
2024-05-11 12:25:19,153 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-11 12:25:19,155 INFO sqlalchemy.engine.